In [1]:
import pandas as pd
import numpy as np
import regex
from pathlib import Path

HRSA_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\hrsa\HRSA - 2021 - Cleaned.csv')
srtr_raw = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\otc-2009-raw.csv')


In [2]:
otc_2021 = HRSA_raw[['OTC Name', 'OTC Address', 'OTC City', 'Common State Name', 'Common State Abbreviation', 'OTC ZIP', 'OTC Longitude', 'OTC Latitude', 'OTC Code']]
otc_2021 = otc_2021.drop_duplicates().reset_index(drop=True)
otc_2021['OTC ZIP'] = otc_2021['OTC ZIP'].astype(object)

In [3]:
srtr_raw['OTC Address'] = srtr_raw['OTC Address'].astype(object)
srtr_raw['Common State Name'] = srtr_raw['Common State Name'].astype(object)
srtr_raw['OTC ZIP'] = srtr_raw['OTC ZIP'].astype(object)

In [4]:
srtr_pending = srtr_raw.loc[srtr_raw['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished = srtr_raw.loc[srtr_raw['OTC Code'].notnull()].reset_index(drop=True)

In [5]:
srtr_pending_manual = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\srtr_pending_manual.csv')

In [6]:
us_hospitals_full = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\us_hospital_locations.csv')

In [7]:
srtr_pending_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].isnull()].reset_index(drop=True)
srtr_finished_2 = srtr_pending_manual.loc[srtr_pending_manual['OTC Code'].notnull()].reset_index(drop=True)

In [8]:
us_hospitals = us_hospitals_full[['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LONGITUDE', 'LATITUDE']]
us_hospitals['OTC Code'] = ''
us_hospitals = us_hospitals.rename(columns={'NAME' : 'OTC Name', 'ADDRESS' : 'OTC Address', 'CITY' : 'OTC City', 'STATE' : 'Common State Abbreviation', 'ZIP' : 'OTC ZIP', 'LONGITUDE' : 'OTC Longtitude', 'LATITUDE' : 'OTC Latitude'})
us_hospitals['key'] = us_hospitals['OTC Name'] + us_hospitals['OTC City'] + us_hospitals['Common State Abbreviation']

C:\Users\DAVIDC~1\AppData\Local\Temp/ipykernel_23464/1547666933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_hospitals['OTC Code'] = ''


In [9]:
srtr_pending_2['OTC Name'] = srtr_pending_2['OTC Name'].str.upper()
srtr_pending_2['OTC City'] = srtr_pending_2['OTC City'].str.upper()
srtr_pending_2['key'] = srtr_pending_2['OTC Name'] + srtr_pending_2['OTC City'] + srtr_pending_2['Common State Abbreviation']


In [10]:
srtr_keys = srtr_pending_2[['key']]
srtr_3 = srtr_keys.merge(us_hospitals, how='left', on='key')

In [11]:
srtr_pending_3 = srtr_3.loc[srtr_3['OTC Name'].isnull()].reset_index(drop=True)
srtr_finished_3 = srtr_3.loc[srtr_3['OTC Name'].notnull()].reset_index(drop=True)

In [12]:
%who

HRSA_raw	 Path	 np	 os	 otc_2021	 pd	 regex	 site	 srtr_3	 
srtr_finished	 srtr_finished_2	 srtr_finished_3	 srtr_keys	 srtr_pending	 srtr_pending_2	 srtr_pending_3	 srtr_pending_manual	 srtr_raw	 
sys	 us_hospitals	 us_hospitals_full	 


In [20]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [21]:
fuzz.ratio('EAST LOS ANGELES DOCTORS HOSPITALLOS ANGELESCA','ALTA BATES MEDICAL CENTERBERKELEYCA')

44

In [22]:
fuzzy = []

for key in srtr_pending_3 :
    name1 = srtr_pending_3.key
    for i,r in us_hospitals.iterrows() :
        name2 = us_hospitals.loc[us_hospitals.key]
    wuzzy = fuzz.ratio(name1, name2)
    if wuzzy > fuzzy :
        fuzzy = wuzzy

KeyError: "None of [Index(['CENTRAL VALLEY GENERAL HOSPITALHANFORDCA',\n       'LOS ROBLES HOSPITAL & MEDICAL CENTER - EAST CAMPUSWESTLAKE VILAGECA',\n       'EAST LOS ANGELES DOCTORS HOSPITALLOS ANGELESCA',\n       'SOUTHERN CALIFORNIA HOSPITAL AT HOLLYWOODHOLLYWOODCA',\n       'KINDRED HOSPITAL BALDWIN PARKBALDWIN PARKCA',\n       'LAKEWOOD REGIONAL MEDICAL CENTERLAKEWOODCA',\n       'MONROVIA MEMORIAL HOSPITALMONROVIACA',\n       'MONTEREY PARK HOSPITALMONTEREY PARKCA',\n       'STAR VIEW ADOLESCENT - P H FTORRANCECA',\n       'CORONA REGIONAL MEDICAL CENTER-MAGNOLIACORONACA',\n       ...\n       'TIDELANDS HEALTH REHABILITATION HOSPITAL AT LITTLE RIVER ANLITTLE RIVERSC',\n       'OWENSBORO HEALTH REGIONAL HOSPITALOWENSBOROKY',\n       'COHEN CHILDREN’S MEDICAL CENTERNEW HYDE PARKNY',\n       'NORTH MS MEDICAL CENTER - EUPORAEUPORAMS',\n       'NORTH MS MEDICAL CENTER - IUKAIUKAMS',\n       'ADVENTIST HEALTHCARE WHITE OAK MEDICAL CENTERSILVER SPRINGMD',\n       'COPPER QUEEN DOUGLAS